In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2

sns.set()

Our library

In [2]:
from proj1_helpers import *
from data_utility import *
from Regressions import *

### Data loading

In [3]:
y_tr, X_tr, ids_tr = load_csv_data('../Data/train.csv')
y_te, X_te, ids_te = load_csv_data('../Data/test.csv')

preliminary Data treatment

In [18]:
n,p = X_tr.shape
keepers = np.ones(p, dtype=bool)

y_tr[y_tr==-1] = 0
y_te[y_te==-1] = 0

# for i in range(p):
#     percentage = len(np.where(X_tr[:,i] ==-999)[0])/n*100
#     if percentage > 70:
#         keepers[i] = 0
keepers[22] = 0 #Remove jet_num

print("Removed features: \n",np.where(keepers == 0))
x_tr = preliminary_treatment_X(X_tr, keepers)
x_te = preliminary_treatment_X(X_te, keepers)
n,p = x_tr.shape

Removed features: 
 (array([22]),)


In [19]:
print(n,p)

250000 33


In [20]:
tx = build_poly(x_tr, 3)
print(tx.shape)

(250000, 100)


## which Definition

In [7]:
which = range(p) #Change according to desired features in tests

# Lasso

In [17]:
a = np.arange(30)+2
np.reshape(a,(1,30))
test = np.vstack((a,a,a,a))
for i in range(4):
    test[i,22]=i
    pass
test = preliminary_treatment_X(test)
test = build_poly(test, 2, pretreated=True)
print(test)

[[  1.   0.   0.   0.   2.   4.   3.   9.   4.  16.   5.  25.   6.  36.
    7.  49.   8.  64.   9.  81.  10. 100.  11. 121.  12. 144.  13. 169.
   14. 196.  15. 225.  16. 256.  17. 289.  18. 324.  19. 361.  20. 400.
   21. 441.  22. 484.  23. 529.  25. 625.  26. 676.  27. 729.  28. 784.
   29. 841.  30. 900.  31. 961.]
 [  0.   1.   0.   0.   2.   4.   3.   9.   4.  16.   5.  25.   6.  36.
    7.  49.   8.  64.   9.  81.  10. 100.  11. 121.  12. 144.  13. 169.
   14. 196.  15. 225.  16. 256.  17. 289.  18. 324.  19. 361.  20. 400.
   21. 441.  22. 484.  23. 529.  25. 625.  26. 676.  27. 729.  28. 784.
   29. 841.  30. 900.  31. 961.]
 [  0.   0.   1.   0.   2.   4.   3.   9.   4.  16.   5.  25.   6.  36.
    7.  49.   8.  64.   9.  81.  10. 100.  11. 121.  12. 144.  13. 169.
   14. 196.  15. 225.  16. 256.  17. 289.  18. 324.  19. 361.  20. 400.
   21. 441.  22. 484.  23. 529.  25. 625.  26. 676.  27. 729.  28. 784.
   29. 841.  30. 900.  31. 961.]
 [  0.   0.   0.   1.   2.   4.   3. 

In [178]:
w = np.zeros(x_tr.shape[1])

In [179]:
lambda_ = 0.5

In [180]:
w, losses = lasso_regression(y_tr, x_tr, lambda_, w, gamma=0.05, max_iters=500, printing=False)
print(w)
print(np.where(w<(5*10**(-5))))

[-0.00382857  1.86734286]
(array([0]),)


In [80]:
def f(b=0,**kwargs):
    print(kwargs.get('a', 0),b)
    pass

f(a=3, b=5)

3 5


In [ ]:
def lasso_regression(y, tx, lambda_, max_iters = 100, gamma = 0.0005, *args, pred=False):
    """implement ridge regression.
    Returns [predictor,] w, loss.
    """
    w = np.zeros(tx.shape[1])
    for n_iter in range(max_iters):
        
        grad,_ = compute_gradient(y, x, w, lambda_=lambda_, loss = "lasso")

        w_old = w
        # update w with gradient update
        w = w - gamma * grad
        # calculate loss
        # y = y.reshape(len(y),1)
        loss = np.mean(abs(y - categories(pred_logistic(x,w))))
        
        #convergence criterion
        if max(abs(w_old-w))/(1+max(abs(w_old))) < 10**-3:
            break  
    
    if pred:
        return linear_predictor, w, loss_f(err_f(y, tx, linear_predictor, w))
    return w, loss_f(err_f(y, tx, linear_predictor, w))